In [1]:
import numpy as np
from scipy.optimize import fsolve

In [2]:
gamma = 1.4

In [3]:
def nu(M): # output: radians
    return np.sqrt((gamma + 1) / (gamma - 1)) * np.arctan(
        np.sqrt(((gamma - 1) / (gamma + 1)) * (M ** 2 - 1))
    ) - np.arctan(np.sqrt(M ** 2 - 1))

In [4]:
# Function to solve for M2
def mach_from_nu(nu_val): # Input:radians
    def func_for_fsolve(M,nu_val):
        return nu_val - nu(M)
    M_init_guess = 3
    M_solution = fsolve(func_for_fsolve,M_init_guess,args=(nu_val,))
    return M_solution[0]

In [5]:
def ratio_P0_P(M):
    return (1 + ((gamma - 1)/2) * M ** 2)**(gamma/(gamma - 1))

In [6]:
# Initial values
M1 = 2.6
theta1 = 5  # in degrees

# Compute nu2
nu2 = nu(M1) + theta1 * np.pi / 180 # radians

In [7]:
M2 = mach_from_nu(nu2)
print(M2)

2.8322924890720786


In [8]:
P2_P1 = ratio_P0_P(M1)/ratio_P0_P(M2)
print(P2_P1)

0.6999509835964826


Oblique Shock

In [9]:
def theta_func(beta, M): # output :radians
    return np.arctan(2 / np.tan(beta) * ((M ** 2 * (np.sin(beta)) ** 2 - 1) /
                                                (M ** 2 * (gamma + np.cos(2 * beta)) + 2)))

In [10]:
def find_beta(theta_deg, M): # Needs input theta in degrees, Output:degrees
    theta = theta_deg*np.pi/180  # Convert deflection angle to radians
    
    def func_for_fsolve(beta, theta, M):
        return theta - theta_func(beta, M)
    beta_initial_guess = np.radians(20)  # Initial guess for beta in radians

    # Solve for beta using fsolve
    beta_solution = fsolve(func_for_fsolve, beta_initial_guess, args=(theta, M))

    # Convert beta to degrees
    beta_deg = np.degrees(beta_solution[0])

    return beta_deg

In [11]:
def NS(M): # Normal Shock Mach Relation
    return ((1 + ((gamma - 1) / 2) * M ** 2)/(gamma * M ** 2 - (gamma - 1) / 2))**0.5

In [12]:
def P_ratio(M): # Normal Shock Pressure ratio 
    return  1+ (2*gamma/(gamma+1)) * (M**2 - 1)   

In [13]:
beta2 = find_beta(theta1, M1)
M_N1 = M1 * np.sin(beta2*np.pi/180)
M_N3 = NS(M_N1)
M3 = M_N3 / np.sin((beta2-theta1)*np.pi/180)
P3_P1 = P_ratio(M_N1)
print(P3_P1)

1.3941570248460025


In [14]:
Cl = (2/(gamma*M1**2)) * (P3_P1 - P2_P1) * np.cos(theta1*np.pi/180)
Cd = (2/(gamma*M1**2)) * (P3_P1 - P2_P1) * np.sin(theta1*np.pi/180)
print(f"Cl = {Cl:.4f}, Cd = {Cd:.4f}")

Cl = 0.1461, Cd = 0.0128


In [15]:
def find_phi(phi):

  #Region 42
  beta42 = find_beta(theta1+phi,M2) # degrees
  M_N2 = M2 * np.sin(beta42*np.pi/180)
  M_N42 = NS(M_N2)
  M4 = M_N42 / np.sin((beta42 - (theta1+phi))*np.pi/180)
  P42_P2 = P_ratio(M_N2)

  #Region 43
  nu43 = nu(M3) + (theta1 + phi)*np.pi/180
  M43 = mach_from_nu(nu43)
  P43_P3 = ratio_P0_P(M3)/ratio_P0_P(M43)

  return P42_P2*P2_P1 - P43_P3*P3_P1

In [16]:
phi_initial_guess = 2
phi = fsolve(find_phi, phi_initial_guess)[0]
print(f"Slip line angle (phi): {phi} degrees")

Slip line angle (phi): 0.002677143714593846 degrees
